In [1]:
pip install torch transformers accelerate


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 91.7 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.4/481.4 kB 55.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.7/781.7 kB 78.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 98.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 13.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2023.4.0
    Uninstalling fsspec-2023.4.0:
      Successfully uninstalled fsspec-2023.4.0

[notice] A new release of pi

In [1]:
import torch

print("PyTorch 버전:", torch.__version__)
print("GPU 사용 가능 여부:", torch.cuda.is_available())

if torch.cuda.is_available():
    print("GPU 이름:", torch.cuda.get_device_name(0))


PyTorch 버전: 2.1.0+cu118
GPU 사용 가능 여부: True
GPU 이름: NVIDIA RTX A4000


In [2]:
import re
import time
import json
from transformers import AutoTokenizer, AutoModelForCausalLM


In [3]:
model_name = "microsoft/phi-3-mini-4k-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name,
                                            torch_dtype=torch.float16)
model.to("cuda")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Phi3ForCausalLM(
  (model): Phi3Model(
    (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
    (layers): ModuleList(
      (0-31): 32 x Phi3DecoderLayer(
        (self_attn): Phi3Attention(
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (qkv_proj): Linear(in_features=3072, out_features=9216, bias=False)
        )
        (mlp): Phi3MLP(
          (gate_up_proj): Linear(in_features=3072, out_features=16384, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (activation_fn): SiLU()
        )
        (input_layernorm): Phi3RMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): Phi3RMSNorm((3072,), eps=1e-05)
        (resid_attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
      )
    )
    (norm): Phi3RMSNorm((3072,), eps=1e-05)
    (rotary_emb): Phi3RotaryEmbedding()
  )
  (lm_head): Linear(in_features=3072, out_features=32064, 

In [18]:
def prompt_a(input_text):
    return f"""
다음 문장을 분석하여 intent, amount, recipient, response를 예시 형식으로 출력해 주세요.

intent는 다음 중 하나입니다: transfer, confirm, cancel, other  
amount는 숫자만 (없으면 None)  
recipient는 사람 이름 등 (없으면 None)  
response는 고객님에게 안내할 자연스러운 한국어 문장

예시:  
text: "아빠한테 오만원 보내줘"  
출력:{{"intent": "transfer","amount": 30000,"recipient": "엄마","response": "엄마님께 30,000원을 송금해드릴까요?"}}  

text: "{input_text}"
"""

def prompt_b(input_text):
    return f"""
다음 문장을 분석하여 intent, amount, recipient, response를 JSON형식으로 출력해 주세요.

intent는 다음 중 하나입니다:  
  transfer: 사용자가 누군가에게 금전을 보내고자 하는 의도,  
  confirm: 이전 발화를 확인하거나 반복하는 표현,  
  cancel: 이전 동작을 취소하거나 거절하려는 의도,  
  inquiry: 송금과 관련된 정보를 묻거나 확인하는 의도,  
  other: 시스템과 무관한 일상 대화 or 분류 불가능한 문장,  
  system_response: 시스템이 사용자에게 재질문하거나 안내하는 응답 발화  
amount는 숫자만 (없으면 None)  
recipient는 사람 이름 등 (없으면 None)  
response는 고객님에게 안내할 자연스러운 한국어 문장, None값이 있다면 꼬리질문을 해서 none값을 채우세요

예시:  
text: "아빠한테 오만원 보내줘"  
{{"intent": "transfer","amount": 30000,"recipient": "엄마","response": "엄마님께 30,000원을 송금해드릴까요?"}}  

text: "{input_text}"
"""

def prompt_c(input_txt):
    return  f"""
    다음 문장을 분석하여 intent, amount, recipient, response를 예시 형식을 따라 추출해 주세요.

    intent는 다음 중 하나입니다: transfer, confirm, cancel, 송금 관련 정보를 물을 시 inquiry,송금 외 질문을 할 시 other
    amount는 숫자만 (없으면 None)
    recipient는 사람 이름 등 (없으면 None)
    response는 고객님에게 안내할 자연스러운 한국어 문장, None값이 있다면 질문을 해서 값을 채울 것

예시:
text: "엄마한테 삼만원 보내줘"

  {{"intent": "transfer","amount": 30000,"recipient": "엄마","response": "엄마님께 30,000원을 송금해드릴까요?"}}


{input_text}
"""


In [19]:
# run_inference.py
def run_inference(prompt: str, tokenizer, model):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    start = time.time()
    outputs = model.generate(**inputs, max_new_tokens=80,do_sample=False)
    end = time.time()

    # 출력 해석
    generated = tokenizer.decode(outputs[0], skip_special_tokens=True)
    output_text = generated.replace(prompt, "").strip()

    print("\n📦 LLM 원문 출력:")
    print(output_text)


    try:
        json_start = output_text.find("{")
        json_end = output_text.rfind("}") + 1
        json_block = output_text[json_start:json_end]
        parsed = json.loads(json_block)
        
    except Exception as e:
        print("JSON 파싱 실패:", e)
        print("📦 원문 출력:", repr(output_text))
        parsed = {}
    
    print(f"\n⏱️ 처리 시간: {round(end - start, 2)}초")
    return parsed


In [20]:
from transformers import AutoTokenizer, AutoModelForCausalLM
# from prompt_templates import prompt_a
# from run_inference import run_inference

# # 1. 모델 로딩
# model_name = "microsoft/phi-3-mini-4k-instruct"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(model_name)
# model.to("cuda")

# 2. 입력 문장
text = "아빠한테 오천 원 보내줘"
prompt = prompt_a(text)

# 3. 단독 인퍼런스 실행
result = run_inference(prompt, tokenizer, model)

print("\n🎯 최종 출력 결과:")
print(result)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 192.00 MiB. GPU 0 has a total capacty of 15.63 GiB of which 189.69 MiB is free. Process 3977112 has 15.44 GiB memory in use. Of the allocated memory 15.23 GiB is allocated by PyTorch, and 13.07 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [9]:
# evaluate_prompt.py

def evaluate_prompt(prompt_func, input_data,tokenizer,model):
    if isinstance(input_data, dict):
        input_data = [input_data]

    results = []
    for s in input_data:
        prompt = prompt_func(s["text"])
        output = run_inference(prompt,tokenizer,model)
        results.append(output)

        # 🔥  결과 출력!
        print(f"\n[입력문장] {s['text']}")
        print("정답:", s["intent"], s.get("slots", {}))
        print("출력:")
        pprint(output)

    def clean(s): return str(s).strip().lower() if s else ""
    def to_int(s): 
        try: return int(str(s).replace(",", "").strip())
        except: return None

    total = len(input_data)

    metrics = {
        "intent_acc": sum(clean(r.get("intent")) == clean(s["intent"]) for r, s in zip(results, input_data)) / total,
        "amount_acc": sum(to_int(r.get("amount")) == s["slots"].get("amount") for r, s in zip(results, input_data)) / total,
        "recipient_acc": sum(clean(r.get("recipient")) == clean(s["slots"].get("recipient")) for r, s in zip(results, input_data)) / total,
        "json_success_rate": sum(bool(r) for r in results) / total
    }

    return {
        "metrics": metrics,
        "results": results
    }


### run inference 작성 완 -> testing 하면 됨
아래는 검토할 것

In [ ]:
# ✅ evaluate_prompt.py

def evaluate_prompt(prompt_func, input_data, tokenizer, model):
    if isinstance(input_data, dict):
        input_data = [input_data]

    results = []
    total_time = 0
    each_times = []

    for s in input_data:
        prompt = prompt_func(s["text"])
        output, elapsed_time = run_inference(prompt, tokenizer, model)
        results.append(output)
        each_times.append(elapsed_time)
        total_time += elapsed_time

        print(f"\n[입력문장] {s['text']}")
        print("🎯 정답:", s["intent"], s.get("slots", {}))
        print("🧠 출력:", output)
        print(f"⏱️ 소요 시간: {elapsed_time}초")

    def clean(s): return str(s).strip().lower() if s else ""
    def to_int(s):
        try: return int(str(s).replace(",", "").strip())
        except: return None

    total = len(input_data)

    metrics = {
        "intent_acc": sum(clean(r.get("intent")) == clean(s["intent"]) for r, s in zip(results, input_data)) / total,
        "amount_acc": sum(to_int(r.get("amount")) == s["slots"].get("amount") for r, s in zip(results, input_data)) / total,
        "recipient_acc": sum(clean(r.get("recipient")) == clean(s["slots"].get("recipient")) for r, s in zip(results, input_data)) / total,
        "json_success_rate": sum(bool(r) for r in results) / total,
        "avg_response_time": round(total_time / total, 2)
    }

    return {
        "metrics": metrics,
        "results": results,
        "times": each_times
    }


# ✅ main.py (사용 예시)

# from transformers import AutoTokenizer, AutoModelForCausalLM
# from prompt_templates import prompt_a, prompt_b
# from evaluate_prompt import evaluate_prompt
# from run_inference import run_inference

# model_name = "microsoft/phi-3-mini-4k-instruct"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(model_name)
# model.to("cuda")

# samples = [
#     {
#         "text": "엄마한테 삼만 원 보내줘",
#         "intent": "transfer",
#         "slots": {"recipient": "엄마", "amount": 30000}
#     },
#     {
#         "text": "그냥 넘어가",
#         "intent": "cancel",
#         "slots": {}
#     }
# ]

# result_a = evaluate_prompt(prompt_a, samples, tokenizer, model)
# result_b = evaluate_prompt(prompt_b, samples, tokenizer, model)

# print("\n📊 프롬프트 성능 비교:")
# for k in result_a["metrics"]:
#     print(f"{k:<18}: A={result_a['metrics'][k]*100:.1f}%   |   B={result_b['metrics'][k]*100:.1f}%")


In [11]:
samples=[
  {
    "text": "철수한테 오천 원 보내줘",
    "intent": "송금",
    "slots": { "recipient": "철수", "amount": 5000 }
  },
  {
    "text": "엄마한테 3만원만",
    "intent": "송금",
    "slots": { "recipient": "엄마", "amount": 30000 }
  },
  {
    "text": "만원만 보내줘",
    "intent": "송금",
    "slots": { "recipient": None, "amount": 10000 }
  },
  {
    "text": "엄마한테 보내줘",
    "intent": "송금",
    "slots": { "recipient": "엄마", "amount": None }
  },
  {
    "text": "나 송금하고 싶어",
    "intent": "송금",
    "slots": { "recipient": None, "amount": None }
  },
  {
    "text": "가스비 내야 하는데 어떻게 해야 해?",
    "intent": "질문",
    "slots": { "topic": "가스비" }
  },
  {
    "text": "내 계좌에 얼마 남았어?",
    "intent": "질문",
    "slots": { "topic": "잔액조회" }
  },
  {
    "text": "그 사람한테 또 보내줘",
    "intent": "송금",
    "slots": { "recipient": "이전대화", "amount": "이전대화" }
  },
  {
    "text": "아까랑 똑같이 보내줘",
    "intent": "송금",
    "slots": { "recipient": "이전대화", "amount": "이전대화" }
  },
  {
    "text": "송금할래",
    "intent": "송금",
    "slots": { "recipient": None, "amount": None }
  },
  {
    "text": "이체 수수료 얼마야?",
    "intent": "질문",
    "slots": { "topic": "이체 수수료" }
  },
  {
    "text": "오늘 아빠랑 점심 먹었어",
    "intent": "기타",
    "slots": {}
  },
  {
    "text": "아빠한테 이만원 보내줘",
    "intent": "송금",
    "slots": { "recipient": "아빠", "amount": 20000 }
  },
  {
    "text": "여보한테 십만 원 이체해줘",
    "intent": "송금",
    "slots": { "recipient": "여보", "amount": 100000 }
  },
  {
    "text": "보내지 마",
    "intent": "취소",
    "slots": {}
  },
  {
    "text": "그냥 넘어가줘",
    "intent": "취소",
    "slots": {}
  },
  {
    "text": "엄마 송금해줘",
    "intent": "송금",
    "slots": { "recipient": "엄마", "amount": None }
  },
  {
    "text": "아, 삼만원 보내는 거였지",
    "intent": "확인",
    "slots": { "amount": 30000 }
  },
  {
    "text": "예금해줘",
    "intent": "기타",
    "slots": { "action": "예금" }
  },
  {
    "text": "누구한테 보낼까요?",
    "intent": "시스템응답",
    "slots": {}
  }
]


In [ ]:
result_a = evaluate_prompt(prompt_a, samples[0])
result_b = evaluate_prompt(prompt_b, samples[0])

print("\n📊 프롬프트 성능 비교:")
for k in result_a["metrics"]:
    print(f"{k:<18}: A={result_a['metrics'][k]*100:.1f}%   |   B={result_b['metrics'][k]*100:.1f}%")


In [28]:
for i,ex in enumerate(sample):
    input_text = ex["text"]
    print(f"\n[{i+1}] 문장: {input_text}")

    result = run_chatbot_inference(input_text) 


[1] 문장: 철수한테 오천 원 보내줘
✅ 파싱된 결과:
{
  "intent": "transfer",
  "amount": 100000,
  "recipient": "철수",
  "response": "철수님께 천원을 송금해드릴까요?"
}

⏱️ 처리 시간: 4.07초

[2] 문장: 엄마한테 3만원만
❌ JSON 파싱 실패: Expecting value: line 1 column 1 (char 0)
📦 원문 출력: 'text: "저는 송금 예정이에요"\n\n  {"intent": "inquiry","amount": None,"recipient": None,"response": "송금 예정이에요네요, 언제 송금'

⏱️ 처리 시간: 3.91초

[3] 문장: 만원만 보내줘
❌ JSON 파싱 실패: Expecting value: line 1 column 52 (char 51)
📦 원문 출력: '{"intent": "transfer","amount": 10000,"recipient": None,"response": "10,000원을 송금하시겠습니까?"}\n\n\n송금하고 싶어요\n\n  {"intent": "'

⏱️ 처리 시간: 3.94초

[4] 문장: 엄마한테 보내줘
✅ 파싱된 결과:
{
  "intent": "transfer",
  "amount": 10000,
  "recipient": "엄마",
  "response": "엄마님께 10,000원을 송금해드릴까요?"
}

⏱️ 처리 시간: 4.09초

[5] 문장: 나 송금하고 싶어
❌ JSON 파싱 실패: Expecting value: line 1 column 1 (char 0)
📦 원문 출력: ''

⏱️ 처리 시간: 4.11초

[6] 문장: 가스비 내야 하는데 어떻게 해야 해?
❌ JSON 파싱 실패: Expecting value: line 1 column 32 (char 31)
📦 원문 출력: '{"intent": "inquiry","amount": None,"recipient": None,"

In [10]:
correct_intent = 0
correct_recipient = 0
correct_amount = 0
    

    pred_intent = result.get("intent")
    pred_recipient = result.get("recipient")
    pred_amount = int(result.get("amount")) if result.get("amount") is not None else None

    # 의도 비교
    if pred_intent == ex["intent"]:
        correct_intent += 1
        print("✔ intent 정답")
    else:
        print(f"❌ intent 오답 (예상: {ex['intent']}, 예측: {pred_intent})")

    # 대상 비교
    if pred_recipient == ex["slots"]["recipient"]:
        correct_recipient += 1
        print("✔ recipient 정답")
    else:
        print(f"❌ recipient 오답 (예상: {ex['slots']['recipient']}, 예측: {pred_recipient})")

    # 금액 비교
    if pred_amount == ex["slots"]["amount"]:
        correct_amount += 1
        print("✔ amount 정답")
    else:
        print(f"❌ amount 오답 (예상: {ex['slots']['amount']}, 예측: {pred_amount})")

# 평가 결과 출력
total = len(sample)
print("\n📊 평가 결과")
print(f"Intent 정확도: {correct_intent}/{total} ({correct_intent/total:.0%})")
print(f"Recipient 정확도: {correct_recipient}/{total} ({correct_recipient/total:.0%})")
print(f"Amount 정확도: {correct_amount}/{total} ({correct_amount/total:.0%})")

❌ intent 오답 (예상: 시스템응답, 예측: None)


KeyError: 'recipient'

In [ ]:
import torch
torch.cuda.empty_cache()
